Archivos utilizados:

dataset_minutes.pkl

df_fechas.pkl


In [ ]:
from google.colab import drive
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa", do_lower_case=True, truncation=True, padding=True, do_basic_tokenize=True, max_length=2024)
model = AutoModelForSequenceClassification.from_pretrained("gtfintechlab/FOMC-RoBERTa", num_labels=3)
config = AutoConfig.from_pretrained("gtfintechlab/FOMC-RoBERTa")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, device=0, framework="pt",
    return_all_scores=True)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
path_minutes='/content/dataset_minutes.pkl'

df=pd.read_pickle(path_minutes)

In [ ]:
df


,Federal_Reserve_Mins
1995-02-01,A meeting of the Federal Open Market Committee...
1995-03-28,A meeting of the Federal Open Market Committee...
1995-05-23,A meeting of the Federal Open Market Committee...
1995-07-06,A meeting of the Federal Open Market Committee...
1995-08-22,A meeting of the Federal Open Market Committee...
...,...
2023-06-14,"The Federal Reserve, the central bank of the U..."
2023-07-26,"The Federal Reserve, the central bank of the U..."
2023-09-20,"The Federal Reserve, the central bank of the U..."
2023-11-01,"The Federal Reserve, the central bank of the U..."


In [ ]:
df.reset_index(inplace=True)
df.columns=['Date', 'Minutes']
df

,Date,Minutes
0,1995-02-01,A meeting of the Federal Open Market Committee...
1,1995-03-28,A meeting of the Federal Open Market Committee...
2,1995-05-23,A meeting of the Federal Open Market Committee...
3,1995-07-06,A meeting of the Federal Open Market Committee...
4,1995-08-22,A meeting of the Federal Open Market Committee...
...,...,...
226,2023-06-14,"The Federal Reserve, the central bank of the U..."
227,2023-07-26,"The Federal Reserve, the central bank of the U..."
228,2023-09-20,"The Federal Reserve, the central bank of the U..."
229,2023-11-01,"The Federal Reserve, the central bank of the U..."


In [ ]:
df.columns = ['Date', 'Federal_Reserve_Mins']
df

,Date,Federal_Reserve_Mins
0,1995-02-01,A meeting of the Federal Open Market Committee...
1,1995-03-28,A meeting of the Federal Open Market Committee...
2,1995-05-23,A meeting of the Federal Open Market Committee...
3,1995-07-06,A meeting of the Federal Open Market Committee...
4,1995-08-22,A meeting of the Federal Open Market Committee...
...,...,...
226,2023-06-14,"The Federal Reserve, the central bank of the U..."
227,2023-07-26,"The Federal Reserve, the central bank of the U..."
228,2023-09-20,"The Federal Reserve, the central bank of the U..."
229,2023-11-01,"The Federal Reserve, the central bank of the U..."


In [ ]:
import pandas as pd

# Resetting the index and renaming columns



# Splitting the sentences in 'Federal_Reserve_Mins' column
df['Federal_Reserve_Mins'] = df['Federal_Reserve_Mins'].apply(lambda x: x.lower().split('. '))

# Creating a new DataFrame to store the results
new_rows = []

# Iterating through the rows of the original DataFrame
for index, row in df.iterrows():
    date = row['Date']
    sentences = row['Federal_Reserve_Mins']

    # Concatenating sentences until the total length exceeds 700 characters
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= 700:
            current_chunk += ". " + sentence if len(current_chunk) > 0 else sentence
        else:
            new_rows.append({'Date': date, 'Federal_Reserve_Mins': current_chunk})
            current_chunk = sentence

    # Adding the last chunk if it exists
    if current_chunk:
        new_rows.append({'Date': date, 'Federal_Reserve_Mins': current_chunk})

# Creating a new DataFrame from the list of new rows
sentences_split = pd.DataFrame(new_rows)




In [ ]:
import pandas as pd
label_mapping = {
    'LABEL_0': 'Dovish',
    'LABEL_1': 'Hawkish',
    'LABEL_2': 'Neutral'
}
def translate_result(result_i):
  return pd.Series({
      label_mapping[dict_i["label"]] : dict_i["score"]
      for dict_i in result_i[0]
  })



In [ ]:
import re
pattern = re.compile(r'\s+')
first_example = pattern.sub(' ', sentences_split.iloc[0, 1][:2500])


In [ ]:
sentences_split['Federal_Reserve_Mins_Sentences'] = sentences_split['Federal_Reserve_Mins'].apply(lambda x: pattern.sub(' ', x).split(". "))

In [ ]:
sentences_split

,Date,Federal_Reserve_Mins,Federal_Reserve_Mins_Sentences
0,1995-02-01,a meeting of the federal open market committee...,[a meeting of the federal open market committe...
1,1995-02-01,"broaddus, forrestal, and parry, presidents ...","[broaddus, forrestal, and parry, presidents of..."
2,1995-02-01,"madigan, associate director, division of ...","[madigan, associate director, division of mone..."
3,1995-02-01,"thornton, assistant vice president, federal ...","[thornton, assistant vice president, federal r..."
4,1995-02-01,as deputy general counsel from the federal res...,[as deputy general counsel from the federal re...
...,...,...,...
99263,2023-12-13,in determining the extent of any additional po...,[in determining the extent of any additional p...
99264,2023-12-13,in assessing the appropriate stance of moneta...,[ in assessing the appropriate stance of monet...
99265,2023-12-13,"goolsbee, patrick harker, philip n. jefferson,...","[goolsbee, patrick harker, philip n. jefferson..."
99266,2023-12-13,it was agreed that the next meeting of the co...,[ it was agreed that the next meeting of the c...


In [ ]:
import numpy as np
def classifier_sentences(text):
    return translate_result(classifier(text,batch_size=128))


In [ ]:
sentences_split.loc[:, ['Dovish', 'Hawkish', 'Neutral']]=sentences_split['Federal_Reserve_Mins_Sentences'].apply(classifier_sentences)

Se han truncado las últimas 5000 líneas del flujo de salida.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101

KeyboardInterrupt: 

In [ ]:

sentences_split.to_pickle("sentences1_prob.pkl")

In [ ]:
path_df_fechas='/content/df_fechas.pkl'
df_fechas=pd.read_pickle(path_df_fechas)

In [ ]:
probabilidades_fechas=sentences_split.groupby("Date")[["Dovish",	"Hawkish",	"Neutral"]].mean()
df_fechas.set_index("Fecha Reunión", inplace=True)
database_df_fechas_y_dhn = pd.merge(df_fechas, probabilidades_fechas, left_index=True, right_index=True)
database_df_fechas_y_dhn.to_pickle("database_df_fechas_y_dhn.pkl")
database_df_fechas_y_dhn

In [ ]:
database_df_fechas_y_dhn.dropna(inplace=True)
database_df_fechas_y_dhn

In [ ]:
sentences_split.groupby("Date")[["Dovish",	"Hawkish",	"Neutral"]].mean().to_pickle("probabilidades_fechas.pkl")
sentences_split.groupby("Date")[["Dovish",	"Hawkish",	"Neutral"]].mean().to_excel("probabilidades_fechas.xlsx")


In [ ]:
sentences_split.groupby("Date")[["Dovish",	"Hawkish", "Neutral"]].mean().plot()

In [1]:
if " " in "pablo ":
  print("hola")

hola
